# FastText

    매커니즘 자체는 Word2Vec 의 확장.
    차이점 : Word2Vec는 단어를 쪼개질 수 없는 단위로 생각.
           FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주.
           -> 내부 단어(subword)를 고려하여 학습한다.

## 1. 내부 단어(subword) 학습
    
    각 단어는 글자 단위 n-gram 의 구성을 취급. 
    시작과 끝을 의미하는 <,> 을 도입하여 벡터로 만들어준다.
    기존 단어에 <,>를 붙인 토큰도 추가해준다.
    n 은 최소값, 최대값 범위를 설정할 수 있다. 기본값은 최소:3, 최대:6
    
    ex) n = 3 , apple
        <ap, app, ppl, ple, le>, <apple> 
        -> 단어들을 가지고 Word2Vec을 수행한다.

## 2. 모르는 단어(Out Of Vocabulary,OOV)

    내부 단어를 통해 OOV 에 대해서도 다른 단어와의 유사도를 계산할 수 있다.


## 3. 단어 집합 내 빈도 수가 적었던 단어(Rare Word)
    
    단어가 희귀 단어라도, 그 단어의 n-gram이 다른 단어의 n-gram과 겹치는 경우,
    비교적 높은 임베딩 벡터값을 가진다. -> 오타가 섞인 단어에 대해서도 일정 수준의 성능을 보인다.
    

## 4. 실습으로 비교하는 Word2Vec vs FastText


In [1]:
# 데이터 전처리
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

targetXML = open('ted_en-20160408.xml','r',encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content> </content> 사이 내용 가져오기.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# content 중간 배경음 부분 제거
content_text = re.sub(r'\([^)]*\)','',parse_text)

# 입력 코퍼스에 대해서 nltk 사용 토큰화 후 구두점 제거, 대문자->소문자
sent_text = sent_tokenize(content_text)

normalized_text = []
for string in sent_text :
    tokens = re.sub(r"[^a-z0-9]+"," ",string.lower())
    normalized_text.append(tokens)

    
result = [word_tokenize(sentence) for sentence in normalized_text]

### 1. Word2Vec

In [2]:
# Word2Vec.

from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format('eng_w2v')

In [3]:
# 없는 단어를 입력하면 에러 발생.
w2v.wv.most_similar('electrofishing')

<ipython-input-3-b20acb32ab81>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  w2v.wv.most_similar('electrofishing')


KeyError: "word 'electrofishing' not in vocabulary"

### 2. FastText

In [4]:
from gensim.models import FastText

fasttext = FastText(result,size=100,window=5,min_count=5,workers=4,sg=1)

In [5]:
# 유사한 단어 계산해서 출력함.
fasttext.wv.most_similar('electrofishing')

[('electrolux', 0.8735015392303467),
 ('electrolyte', 0.8697566390037537),
 ('electro', 0.8612461090087891),
 ('electroshock', 0.8494871854782104),
 ('electroencephalogram', 0.8427205085754395),
 ('electrogram', 0.836060643196106),
 ('electrochemical', 0.8309044241905212),
 ('electrode', 0.8255390524864197),
 ('electron', 0.8252688050270081),
 ('electrons', 0.8225287199020386)]